In [1]:
import pandas as pd
import base64
import requests
import os
from glob import glob
import re
import time
import pickle
# OpenAI API Key
api_key = "YOUR_API_KEY"
file_name = 'baroque'

In [2]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
# Function to call OpenAI's API
def analyze_image(base64_image):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Please describe the elements in the image and how it makes you feel, as if you were describing the image to someone who had never seen it before."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 4096
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

In [4]:
curr_path = os.getcwd()
csv_file_path = os.path.join(curr_path,file_name+'.csv')
df = pd.read_csv(csv_file_path)

In [5]:
image_folder_path= os.path.join(curr_path, file_name)
image_paths = [os.path.join(image_folder_path, img_name) for img_name in df['image']]
image_base = df['image'].tolist()
for i in df['image']:
    print(i)
print(len(image_paths))


Cuyp__Aelbert__Young_Herdsmen_with_Cows__ca__1655_60_.jpg
Jordaens__Jacob__The_King_Drinks__ca__1645_50_.jpg
Joseph_Telling_His_Dreams___Rembrandt_van_Rijn_.jpg
Kalf__Willem__Still_Life_with_Fruit__Glassware__and_a_Wanli_Bowl__1659_.jpg
Kalf__Willem__Still_Life_with_Silver_Jug__Around_1655___Around_1657_.jpg
King_Solomon_Receiving_the_Queen_of_Sheba___Frans_Francken_II_.jpg
Lairesse__Gerard_de__Hiems__Winter___ca__1680_.jpg
Lairesse__Gerard_de__Joseph_Reveals_Himself_to_His_Brothers__1641_1711_.jpg
Lairesse__Gerard_de__Odysseus_bij_Calypso__ca__1680_.jpg
Lairesse__Gerard_de__Perseus_met_het_hoofd_van_Medusa__1682_.jpg
Landscape_with_a_Calm_.jpg
Lanfranco__Giovanni__Angélica_e_Medoro__1605_1647_.jpg
Large_Self_Portrait___Rembrandt_Harmenszoon_van_Rijn_.jpg
Lastman__Pieter__David_Gives_Uriah_a_Letter_for_Joab__1619_.jpg
Lastman__Pieter__Jonah_and_the_Whale__1621_.jpg
Lely__Peter__Nymphs_by_a_Fountain__Early_1650s_.jpg
Leone__Andrea_di__Tobit_Burying_the_Dead__1640s_.jpg
Leyster__Judith__

In [7]:
idx = 0

if not os.path.exists('./temp_'+file_name+'.pickle'):
    res_dct = {'image': []  ,'descript':[]}
else:
    with open('./temp_'+file_name+'.pickle','rb') as f:
        res_dct = pickle.load(f)
if not(idx==0):
    image_paths = image_paths[idx:]

for img in image_paths:
    res = analyze_image(encode_image(img))
    str = res['choices'][0]['message']['content']
    new_str = re.sub("\n", " ", str)
    res_dct['image'].append(image_base[idx])
    res_dct['descript'].append(new_str)
    df = pd.DataFrame(res_dct)
    df.to_csv('./'+file_name+'.csv')
    with open('./temp_'+file_name+'.pickle','wb') as f:
        pickle.dump(res_dct, f, protocol=pickle.HIGHEST_PROTOCOL)
    idx = len(res_dct['descript'])
    time.sleep(13)
    print(res_dct['image'][-1],res_dct['descript'][-1],sep = ' | ')

Metsu__Gabriël__The_Sick_Child__Around_1663___Around_1664_.jpg | You are looking at a painting where the focus is on two figures, likely a mother and child. The mother is seated and the child rests in her lap looking slightly to the side, out towards the viewer. The woman gazes affectionately at the child, her hands softly positioned on the child's arm and leg. She is dressed in garments that suggest a historical setting with subdued colors: a dark brown dress with a white collar and a black headpiece. The child wears a bright orange-yellow shirt and white undergarments, creating a vivid contrast with the mother's attire.  At the bottom left of the painting, there's a small, plain wooden table with a simple brown earthenware pot. The background is quite minimal, with an off-white wall and a small window or opening showing a dark, muted scene of a figure amidst trees. Above the woman, there's a discolored patch on the wall, with a faint, unidentifiable marking. To the right, a dark gree

In [8]:
max_length = max(len(lst) for lst in res_dct.values())
for key in res_dct:
    while len(res_dct[key]) < max_length:
        res_dct[key].append('NaN')

df = pd.DataFrame(res_dct)
df.to_csv('./tag/'+file_name+'_tag.csv',index = False)